In [22]:
import json
import operator
import os
import requests

from datetime import datetime
from dotenv import load_dotenv
from pyairtable import Api
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [23]:
# Env Variables
load_dotenv()

AIRTABLE_TOKEN = os.getenv("AIRTABLE_TOKEN")
AIRTABLE_TOKEN_ID = os.getenv("AIRTABLE_TOKEN_ID")
AIRTABLE_BASE_ID = os.getenv("AIRTABLE_BASE_ID")
SETS_TABLE_ID = os.getenv('TCGRIPLYTICS_AIRTABLE_SETS_ID')
CARDS_TABLE_ID = os.getenv('TCGRIPLYTICS_AIRTABLE_CARDS_ID')

In [24]:
# Run variables
set_id_jpn = '332'
set_uuids = [32827430, 6055010, 96588164, 89354591, 11592280, 75404647, 2148733, 75959076, 34153438]
card_language = 'JPN'

In [25]:
class CardSet:
    def __init__(self, uuid, name, set_code, source_url, image_url, language, release_year, release_date, epoch, card_count, printed_count):
        self.uuid = uuid
        self.name = name
        self.set_code = set_code
        self.source_url = source_url
        self.image_url = image_url
        self.language = language
        self.release_year = release_year
        self.release_date = release_date
        self.release_epoch = epoch
        self.card_count = card_count
        self.printed_count = printed_count

    def __str__(self):
        return (f"Set UUID: {self.uuid}, Set Name: {self.name}, Set Code: {self.set_code}, CardSet(Source URL: {self.source_url}"
                f"Image URL: {self.image_url}, Language: {self.language}, Release Year: {self.release_year}, Release Date: {self.release_date}"
                f"Release Epoch: {self.release_epoch}, Card Count: {self.card_count}, Printed Count: {self.printed_count})")

    def update_card_count(self, new_card_count):
        self.card_count = new_card_count

    def update_printed_count(self, new_printed_count):
        self.printed_count = new_printed_count

    def update_release_date(self, new_release_date):
        self.release_date = new_release_date

    # Add more methods as needed


In [35]:
class Card:
    def __init__(self, sequence_number, name, set_name, rarity, supertype, subtypes, printed_number, language, artist, image_url, card_url, set_uuid, card_uuid):
        self.sequence_number = sequence_number
        self.name = name
        self.set_name = set_name
        self.rarity = rarity
        self.supertype = supertype
        self.subtypes = subtypes
        self.printed_number = printed_number
        self.language = language
        self.artist = artist
        self.image_url = image_url
        self.card_url = card_url
        self.set_uuid = set_uuid
        self.card_uuid = card_uuid

    def __str__(self):
        return (f"Sequence Number: {self.sequence_number}, Card Name: {self.name}, Set Name: {self.set_name}, Rarity: {self.rarity}, Supertype: {self.supertype}, "
        f"Subtypes: {self.subtypes}, Printed Number: {self.printed_number}, Language: {self.language}, Artist: {self.artist}, "
        f"Image URL: {self.image_url}, Card URL: {self.card_url}, Set UUID: {self.set_uuid}, Card UUID: {self.card_uuid})")



In [27]:
# Function to convert date to epoch
def convert_date_to_epoch(date_str):
    for fmt in ("%B %d, %Y", "%B %Y", "%b %d, %Y"):
        try:
            date_obj = datetime.strptime(date_str, fmt)
            return int(date_obj.timestamp())
        except ValueError:
            continue
    raise ValueError(f"Date format for '{date_str}' is not recognized")

In [28]:
# Get Set
# API - https://jpn-cards-site.readthedocs.io/en/latest/home/
def load_jpn_set(set_id_jpn):
    # Define the URL of the API endpoint
    url = f'https://www.jpn-cards.com/v2/set/{set_id_jpn}'
    #url = 'https://www.jpn-cards.com/v2/set/'  # Get All Sets
    
    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        #print("Response Data:", data)
        return data
    else:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")

# cardSet = load_jpn_set(set_id_jpn)
# print(json.dumps(cardSet, indent=4))
# print(cardSet['id'])

def load_all_jpn_sets():
    # Define the URL of the API endpoint
    url = 'https://www.jpn-cards.com/v2/set/'
    
    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        #print("Response Data:", data)
        return data
    else:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")

# cardSet = load_jpn_set(set_id_jpn)
# print(json.dumps(cardSet, indent=4))
# print(cardSet['id'])

def load_jpn_set_by_uuid(set_uuid):
    # Define the URL of the API endpoint
    url = f'https://www.jpn-cards.com/v2/set/uuid/{set_uuid}'
    
    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        #print("Response Data:", data)
        return data
    else:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")

# cardSet = load_jpn_set(set_id_jpn)
# print(json.dumps(cardSet, indent=4))
# print(cardSet['id'])


In [ ]:
# ALL SETS
all_sets = load_all_jpn_sets()
print(all_sets)

In [ ]:
# ALL SETS
#print(json.dumps(all_sets, indent=4))
#newlist = sorted(all_sets, key=itemgetter('date'), reverse=True)

card_sets = []
for item in all_sets:
    #print(item)
    item['epoch'] = convert_date_to_epoch(item['date'])

sorted_data = sorted(all_sets, key=lambda x: x['epoch'])
for item in sorted_data:
    card_sets.append(item['id'])

print(card_sets)
print(len(card_sets))

#print(json.dumps(sorted_data, indent=4))

In [29]:
# Get all cards from set
def get_cards_from_set(set_id_jpn):
    # Define the URL of the API endpoint
    url = f'https://www.jpn-cards.com/v2/card/set_id={set_id_jpn}'
    
    # Make the GET request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        #print("Response Data:", data)
        return data
    else:
        print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")

# cards = get_cards_from_set(set_id_jpn)
#print(json.dumps(cards, indent=4))

In [30]:
def createCardSetRecord(cardSet, Api):
    print(cardSet)
    table = Api.table(AIRTABLE_BASE_ID, SETS_TABLE_ID)
    table.create({'Name': cardSet['name'], 'Release Epoch': cardSet['release_epoch'], 'Language': cardSet['language'], 
                  'Set Card Count': cardSet['card_count'], 'Printed Card Count': cardSet['printed_count'], 'Set Code': cardSet['set_code'], 
                  'Source URL': cardSet['source_url'], 'Image URL': cardSet['image_url'], 'Set UUID': cardSet['uuid']}, typecast=True)

In [38]:
def createCardRecord(card, cardSet, Api):
    print(card)
    print(card.name)
    table = Api.table(AIRTABLE_BASE_ID, CARDS_TABLE_ID)
    table.create({'Set Sequence Number': card.sequence_number, 'Name': card.name, 'Set Name': card.set_name, 'Rarity': card.rarity, 'Supertype': card.supertype, 
                  'Subtypes': card.subtypes, 'Printed Number': card.printed_number, 
                  'Language': card.language, 'Artist': card.artist, 'Image URL': card.image_url, 'Card URL': card.card_url, 
                  'Set UUID': cardSet['uuid'], 'Card UUID': card.card_uuid}, typecast=True)

In [ ]:
# ALL SETS
# Load new Set Record into AirTable
api = Api(AIRTABLE_TOKEN)

for card_set in card_sets:
    print(card_set)
    cardSet = load_jpn_set(card_set)
    print(cardSet)
    cards = get_cards_from_set(card_set)
    #cardSet['card_count'] = len(cards['data'])
    #cardSet['release_epoch'] = convert_date_to_epoch(cardSet['date'])
    #createCardSetRecord(cardSet, api)

    new_cards = []

    for card in cards['data']:
        subtypes = []
        for subtype in card['subtypes']:
            subtypes.append(subtype)
        print("CARD SET: ", cardSet['uuid'])
        print("CARD: ", card['uuid'])
        newCard = Card(card['name'], card['setData']['name'], card['rarity'], card['supertype'], subtypes, card['sequenceNumber'], 
                    card['printedNumber'], card_language, card['artist'], card['imageUrl'], card['cardUrl'], cardSet['uuid'], card['uuid'])
        
        new_cards.append(newCard)

    print("Number of cards in set:", len(new_cards))

    for card in new_cards:
        createCardRecord(card, cardSet, api)

In [41]:
# UUID Sets
# Load new Set Record into AirTable
uuid_sets = []
card_sets = []
api = Api(AIRTABLE_TOKEN)

for set_uuid in set_uuids:
    cardSetUUID = load_jpn_set_by_uuid(set_uuid)
    uuid_sets.append(cardSetUUID)

for uuid_set in uuid_sets:
    uuid_set['epoch'] = convert_date_to_epoch(uuid_set['date'])

sorted_data = sorted(uuid_sets, key=lambda x: x['epoch'])
for item in sorted_data:
    card_sets.append(item['id'])

for card_set in card_sets:
    print(card_set)
    cardSet = load_jpn_set(card_set)
    load_jpn_set_by_uuid(set_uuid)
    print(cardSet)
    cards = get_cards_from_set(card_set)
    #cardSet['card_count'] = len(cards['data'])
    #cardSet['release_epoch'] = convert_date_to_epoch(cardSet['date'])
    #createCardSetRecord(cardSet, api)

    new_cards = []

    for card in cards['data']:
        subtypes = []
        for subtype in card['subtypes']:
            subtypes.append(subtype)
        print("CARD SET: ", cardSet['uuid'])
        print("CARD: ", card['uuid'])
        #newCard = Card(card['name'], card['setData']['name'], card['rarity'], card['supertype'], subtypes, card['sequenceNumber'], 
                    #card['printedNumber'], card_language, card['artist'], card['imageUrl'], card['cardUrl'], cardSet['uuid'], card['uuid'])
        
        newCard = Card(card['sequenceNumber'], card['name'], card['setData']['name'], card['rarity'], card['supertype'], subtypes, 
            card['printedNumber'], card_language, card['artist'], card['imageUrl'], card['cardUrl'], cardSet['uuid'], card['uuid'])
        
        new_cards.append(newCard)

    print("Number of cards in set:", len(new_cards))

    for card in new_cards:
        createCardRecord(card, cardSet, api)
    


24
{'id': 24, 'name': 'Blue Sky Stream', 'source_url': 'https://www.tcgcollector.com/cards/jp/blue-sky-stream', 'image_url': 'https://static.tcgcollector.com/content/images/a3/cc/27/a3cc27411c1f1b89728a129ccc1ffcbc2a698855b5966375ce61a0298c77df48.png', 'language': 'JPN', 'year': '2021', 'date': 'July 9, 2021', 'card_count': 90, 'printed_count': 67, 'set_code': 's7r', 'uuid': 34153438}
CARD SET:  34153438
CARD:  55534947
CARD SET:  34153438
CARD:  75176824
CARD SET:  34153438
CARD:  6960531
CARD SET:  34153438
CARD:  60296171
CARD SET:  34153438
CARD:  95925636
CARD SET:  34153438
CARD:  39100348
CARD SET:  34153438
CARD:  63938023
CARD SET:  34153438
CARD:  15130762
CARD SET:  34153438
CARD:  55820096
CARD SET:  34153438
CARD:  38220315
CARD SET:  34153438
CARD:  97622385
CARD SET:  34153438
CARD:  54381700
CARD SET:  34153438
CARD:  34307315
CARD SET:  34153438
CARD:  46778280
CARD SET:  34153438
CARD:  87601704
CARD SET:  34153438
CARD:  84900586
CARD SET:  34153438
CARD:  11955623
C

In [ ]:
new_cards = []

for card in cards['data']:
    subtypes = []
    for subtype in card['subtypes']:
        subtypes.append(subtype)
    print("CARD SET: ", cardSet['uuid'])
    print("CARD: ", card['uuid'])
    newCard = Card(card['name'], card['setData']['name'], card['rarity'], card['supertype'], subtypes, card['sequenceNumber'], 
                   card['printedNumber'], card_language, card['artist'], card['imageUrl'], card['cardUrl'], cardSet['uuid'], card['uuid'])
    
    new_cards.append(newCard)

print("Number of cards in set:", len(new_cards))

In [ ]:
for card in new_cards:
    createCardRecord(card, cardSet, api)

In [ ]:
class Watcher:
    DIRECTORY_TO_WATCH = "path/to/your/directory"

    def __init__(self):
        self.observer = Observer()

    def run(self):
        event_handler = Handler()
        self.observer.schedule(event_handler, self.DIRECTORY_TO_WATCH, recursive=False)
        self.observer.start()
        try:
            while True:
                time.sleep(5)
        except:
            self.observer.stop()
            print("Observer Stopped")
        self.observer.join()
